# Programmation SAT - Générateur de championnat

## Exercice 2 :

Question 1 :

Le nombre de variables propositionnelles utilisées est obtenue en prenant pour j la valeur nj, x = ne et y = ne, soit :
nj*ne²+ne²+ne+1

In [1]:
def codage(ne,nj,j,x,y):
    return (j*ne*ne + x*ne + y +1)

In [2]:
def decodage(k,ne,nj):
    """Retourne le tuple (j,x,y)"""
    j = (k-1)//(ne*ne)
    x = ((k-1)%(ne*ne))//ne
    y = (k-1)%ne
    return (j,x,y)

In [3]:
#codage(15,15,5,1,8)

In [4]:
#decodage(1149,15,15)

## Exercice 3 :

Partie 1 :

In [5]:
def clauseAuMoins(var):
    """ Prend en argument une liste d'entiers et renvie une clause au
    format DIMACS correspondant à la contrainte :
    au moins une de ces variables est vraie
    On suppose qu'on rentre une séquence valide de variables"""
    
    s = ""
    for i in var :
        s += str(i) + " "
    s += "0\n"
    return s

In [6]:
print(clauseAuMoins([1,3,4]))

1 3 4 0



In [7]:
def clauseAuPlus(var):
    """Prend en argument une liste d'entiers et renvoie les clauses au
    format DIMACS, correspondant à la contrainte :
    au plus une de ces variables est vraie"""
    
    s = ''
    for i in range(len(var)):
        for j in range(i+1,len(var)):
            s += clauseAuMoins([-var[i],-var[j]])
    return s
            

In [8]:
print(clauseAuPlus([1,2,3]))

-1 -2 0
-1 -3 0
-2 -3 0



Partie 2 :

1- La contrainte C1 peut être traduite de la manière suivante :
Pour tout x appartenant à [0,ne], pour tout j appartenant à [0,nj]
	La somme pour tous les y appartenant à [0,ne] \ {x} du codage Mj,x,y doit être inférieure ou égale à 1

In [14]:
def encoderC1(ne,nj):
    """ Rend la contrainte C1 :
    chaque équipe ne peut jouer plus d'un match par jour"""
    s = ''
    for x in range(ne):
        for j in range(nj):
            value = [codage(ne,nj,j,x,y) for y in range(ne) if y!=x]
            value += [codage(ne,nj,j,y,x) for y in range(ne) if y!=x]
            s += clauseAuPlus(value)
    return s

In [143]:
#print(encoderC1(3,4))

3-Le nombre de contraintes générées pour 3 équipes sur 4 jours est de 12.
En effet, il n'y a que 3 combinaisons possibles pour les paires d'équipes : (0,1) ; (0,2) ; (1,2)
et cela pour 4 jours différents, ce qui fait bien 12.

4-La contrainte C2 peut se traduire de cette façon :
Pour tout x appartenant à [0,ne], Pour tout y appartenant à [0,ne] \ {x} :
	La somme pour tous les j appartenant à [0,nj]  du codage Mj,x,y  vaut 1
	Et la somme pour tous les j appartenant à [0,nj]  du codage Mj,y,x  vaut 1

In [144]:
def encoderC2(ne,nj):
    """Rend la contrainte C2 :
    sur la durée du championnat, il y a exactement 2 matchs par équipe adverse"""
    s = ''
    for x in range(ne):
        for y in range(x+1,ne):
            if(x!=y):
                value = [codage(ne,nj,j,x,y) for j in range(nj)]
                s+=clauseAuMoins(value)
                s+=clauseAuPlus(value)
                value = [codage(ne,nj,j,y,x) for j in range(nj)]
                s+=clauseAuMoins(value)
                s+=clauseAuPlus(value)
    return s

In [145]:
#print(encoderC2(3,4))


6-Il y a 42 contraintes générées. En effet, pour traduire qu'il y a bien un seul match entre deux équipes il faut les deux clauses au plus et au moins. La clause au moins se traduit en une seule contrainte, et la clause au plus se traduit en 6 contraintes car un match entre deux équipes a la possibilité d'être jouer durant un des quatres. Il faut ensuite traduire cela pour l'inverse (si le match a été joué à domicile, il faut aussi un match à l'extérieur entre ces deux équipes et inversement). Cela nous donne bien 42 contraintes.

In [146]:
def encoder(ne,nj):
    "Encode toutes les contraintes C1 et C2 pour ne et nj donné"
    c1 = encoderC1(ne,nj)
    cpt = 0
    i = 0
    while i in range(len(c1)) :
        if(c1[i]=='\n'):
            cpt+=1
        i+=1
    c2 = encoderC2(ne,nj)
    i = 0
    while i in range(len(c2)) :
        if(c2[i]=='\n'):
            cpt+=1
        i+=1
    s = 'p cnf '+ str(nj*ne*ne+ne*ne+ne+1) + ' '+ str(cpt)+'\n'
    s+=c1
    s+=c2
    return s

In [149]:
#print(encoder(3,4))

Partie 3 :

Glucose donne la réponse suivante pour ne = 3 et nj = 4.

s UNSATISFIABLE

Partie 4 :

In [150]:
def decoder(fichier,fichierNomEquipe,ne,nj,affichage=True):
    """A partir d'un fichier contenant la sortie de l'appel à glucose 
    traduit le modèle en une solution de planning des matchs """
    f = open(fichierNomEquipe, "r")
    equipes = []
    nom = ''
    for i in f.read():
        if(i != '\n'):
            nom += i
        else:
            equipes.append(nom)
            nom = ''
    sol = open(fichier,'r')
    s = sol.readlines()
    if(s[0][1]  == '\\'):
        i = 0
        li = []
        ligne=''
        while(i<len(s[0])):
            if(s[0][i]=='\\' and s[0][i+1]=='n'):
                li.append(ligne)
                ligne =''
                i+=2
            else:
                ligne+=s[0][i]
                i+=1
        s = li
    satisOrNot = s[len(s)-2]
    solution = s[len(s)-1]
    if(s[len(s)-1][2]== "U"):
        print("Aucun modèle satisfaisant ces contraintes n'a été trouvé")
        return 0
    else:
        s = []
        solution = solution.split()
        for i in range(1,len(solution)-1):
            if(solution[i][0]!='-'):
                s.append(i)
        if(affichage):
            for i in s:
                (j,x,y) = decodage(i,ne,nj)
                print("Match entre l'équipe",equipes[x],"jouant à domicile, et l'équipe",equipes[y],"au jour",j)
        return len(s)
    f.close()
    sol.close()

In [130]:
decoder('solution_3_4.txt',"nomEquipes.txt",3,4,affichage=False)

Aucun modèle satisfaisant ces contraintes n'a été trouvé


0

Partie 5 :

In [139]:
import os
import subprocess
def assemblage(ne,nj,fichierNomEquipe,affichage=True):
    """Etant donné ne et nnj et un fichier de nom d'équipes,
    affiche un planning des matchs"""
    s = encoder(ne,nj)
    title = "clause_"+str(ne)+'_'+str(nj)+'.cnf'
    fichier = open(title, "w")
    fichier.write(encoder(ne,nj))
    fichier.close()
    result = subprocess.run(['./glucose_static','-model',title], stdout=subprocess.PIPE)
    result = str(result.stdout)
    result = result[2:]
    title = "solution_"+str(ne)+'_'+str(nj)+".txt"
    fichier = open(title,"w")
    fichier.write(result)
    fichier.close()
    result = decoder(title,fichierNomEquipe,ne,nj,affichage)
    subprocess.run(['rm',title], stdout=subprocess.PIPE)
    return result

In [140]:
assemblage(3,6,"nomEquipes.txt")

Match entre l'équipe Italie jouant à domicile, et l'équipe France au jour 0
Match entre l'équipe France jouant à domicile, et l'équipe Italie au jour 1
Match entre l'équipe Italie jouant à domicile, et l'équipe Allemagne au jour 2
Match entre l'équipe France jouant à domicile, et l'équipe Allemagne au jour 3
Match entre l'équipe Allemagne jouant à domicile, et l'équipe Italie au jour 4
Match entre l'équipe Allemagne jouant à domicile, et l'équipe France au jour 5


6

# Exercice 4 : Optimisation du nombre de jours

In [142]:
for ne in range(3,11):
    for nj in range(ne*(ne-1),100):
        result = assemblage(ne,nj,"nomEquipes.txt",False)
        if(result > 0):
            print("Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour",ne,"équipes est",result)
            break

Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 3 équipes est 6
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 4 équipes est 12
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 5 équipes est 20
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 6 équipes est 30
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 7 équipes est 42
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 8 équipes est 56
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 9 équipes est 72
Le nombre minimal de jours nj pour pouvoir planifier tous les matchs pour 10 équipes est 90


# Exercice 5 : Extention : équilibrer les déplacements et les week-ends

Partie 1 :

La contrainte 1-a) peut s'écrire de la forme suivante :
Pour tout x appartenant à [0,ne], pour tout y appartenant à [0,ne]\\{x}

    La somme sur j1 tel que j1 appartient à [0,nj] et j1%2 = 1 de
    || Mj1,y,x || >= somme sur j tel que j appartient à [0,nj] de || Mj,y,x || * pext

    Cela revient donc à :
    La somme sur j1 tel que j1 appartient à [0,nj] et j1%2 = 1 de
    - || Mj1,y,x || <= nj - (somme sur j tel que j appartient à [0,nj] de || Mj,y,x || * pext)


La contrainte 1-b) peut s'écrire de la forme suivante :
Pour tout x appartenant à [0,ne], pour tout y appartenant à [0,ne]\\{x}

    La somme sur j1 tel que j1 appartient à [0,nj] et j1%2 = 1 de
    || Mj1,x,y || >= somme sur j tel que j appartient à [0,nj] de || Mj,x,y || * pdom

    Cela revient donc à :
    La somme sur j1 tel que j1 appartient à [0,nj] et j1%2 = 1 de
    - || Mj1,x,y || <= nj - (somme sur j tel que j appartient à [0,nj] de || Mj,x,y || * pdom)

La contrainte 2-a) peut s'écrire de la forme suivante :
Pour tout x appartenant à [0,ne],
    pour tout y appartenant à [0,ne]\\{x},
            pour tout j appartenant à [0,nj-2]

                Somme sur i, commençant à i = j et se terminant à i = j +2 || M,i,x,y || <= 2


La contrainte 2-b) peut s'écrire de la forme suivante :
Pour tout x appartenant à [0,ne],
    pour tout y appartenant à [0,ne]\\{x},
            pour tout j appartenant à [0,nj-2]

                Somme sur i, commençant à i = j et se terminant à i = j +2 || M,i,y,x || <= 2


Partie 2 :

